In [79]:
import numpy as np
import faiss
from langchain_openai import OpenAIEmbeddings
# from langchain.embeddings import FakeEmbeddings

DATA

In [80]:
#this is where you would input all the existing terms

ab_initio = "Ab initio is a Latin term that means \"from the beginning” or “from inception.” Ab initio is used to indicate that some fact existed from the start of a relevant time period. It is often used as part of the phrase “void ab initio,” meaning something - such as a marriage - was void from the beginning."
ar = "A/R is the abbreviation for accounts receivable."
aka = "A.K.A. is the abbreviation of “also known as.” It is often used to specify a party’s aliases or nicknames in a case."
aba = "ABA is an abbreviation for American Bar Association (ABA)."
abandon = "Abandon refers to the act of abandonment. A party can “abandon” in multiple fields of law and abandoning generally, though not always, involves voluntarily giving up a right."
abet = "Abet refers to criminally assisting another person in the commission of a crime including planning a crime, escaping from a crime, or in the actual commission of the crime. A party that abets commission of a crime can be charged and prosecuted on grounds of accomplice liability. Under federal law, parties charged with abetting are charged as principals and are, therefore, subject to the same penalties as if they had physically committed the crime themselves. A party who abets before a crime occurs is frequently known as an accessory before the fact. A party who abets by nature of being present at the scene of the crime like a lookout is frequently known as a principal in the second decree. In statutes or case law, abetting is also frequently written as “aiding and abetting”."
abeyance = "An abeyance is a temporary suspension of activity while awaiting the resolution of some other proceeding without which the activity in abeyance cannot continue. In the case of a plea in abeyance, the defendant enters a conditional plea of guilty which is then held in abeyance subject to a set of conditions. If the defendant fulfills these conditions, the charges are dropped. If the defendant fails to complete the conditions, the guilty plea is entered. For example, a defendant may be required to undergo a drug addiction treatment program or have a guilty plea entered. Appeals can also be held in abeyance, usually to await the outcome of another court case which concerns similar issues. In property law, abeyance occurs when the true owner of property cannot immediately be determined. In these scenarios, that property is held in abeyance with no one able to exert ownership rights until the true owner can be determined."
texts = [ab_initio, ar, aka, aba, abandon, abet, abeyance, award]

In [81]:
# Names is the name of the legal term
# tag_names (subject's names) are all tag names exisitng for current terms

names = ["ab initio", "A/R", "a.k.a.", "ABA", "abandon", "abet", "abeyance"]
definition = texts
tag_names = [
    [ "ACADEMIC TOPICS", "legal education and writing", "COMMERCE", "commercial activities", "business law", "contracts", "LIFE EVENTS", "family & personal matters", "divorce", "marriage", "THE LEGAL PROCESS", "criminal law", "criminal procedure", "civil procedure", "legal theory", "business sectors", "courts and procedure", "criminal law and procedure", "family law", "legal education and practice", "wex definitions" ],
    ["COMMERCE", "commercial activities", "accounting", "finance", "financial services", "business", "law", "business organizations", "corporations", "business sectors", "commercial transactions", "money and financial problems", "tax", "wex definitions" ],
    [ "legal education and writing", "THE LEGAL PROCESS", "courts", "criminal law", "criminal procedure", "legal practice/ethics", "civil procedure", "accidents and injuries", "courts and procedure", "criminal law and procedure", "legal education and practice", "wex definitions" ],
    [ "ACADEMIC TOPICS", "legal education and writing", "THE LEGAL PROCESS", "legal practice/ethics", "legal education and practice", "wex definitions" ],
    [ "LIFE EVENTS", "standards of tort liability", "family & personal matters", "divorce", "marriage", "PROPERTY", "land use & zoning law", "property & real estate law", "trusts, inheritances & estates", "trademark law", "THE LEGAL PROCESS", "accidents and injuries", "courts and procedure", "family law", "legal education and practice", "property law", "wex definitions" ],
    [ "THE LEGAL PROCESS", "criminal law", "statutory interpretation", "criminal law and procedure", "wex definitions" ],
    [ "PROPERTY", "property & real estate law", "property law", "wex definitions" ]
]

In [82]:
# Making a dicitionary of all the legal terms for each tag/subject

# Initialize an empty dictionary to store tags as keys and names as values.
tag_dictionary = {}

# Iterate over each name and its corresponding tag list.
for i, name in enumerate(names):
    for tag in tag_names[i]:
        # If the tag is already in the dictionary, append the name to its list.
        if tag in tag_dictionary:
            tag_dictionary[tag].append(name)
        # Otherwise, create a new entry in the dictionary with the name in a list.
        else:
            tag_dictionary[tag] = [name]

# Now `tag_dictionary` is in the format {tag_name: [all of the names under that tag]}
print(tag_dictionary)

{'ACADEMIC TOPICS': ['ab initio', 'ABA'], 'legal education and writing': ['ab initio', 'a.k.a.', 'ABA'], 'COMMERCE': ['ab initio', 'A/R'], 'commercial activities': ['ab initio', 'A/R'], 'business law': ['ab initio'], 'contracts': ['ab initio'], 'LIFE EVENTS': ['ab initio', 'abandon'], 'family & personal matters': ['ab initio', 'abandon'], 'divorce': ['ab initio', 'abandon'], 'marriage': ['ab initio', 'abandon'], 'THE LEGAL PROCESS': ['ab initio', 'a.k.a.', 'ABA', 'abandon', 'abet'], 'criminal law': ['ab initio', 'a.k.a.', 'abet'], 'criminal procedure': ['ab initio', 'a.k.a.'], 'civil procedure': ['ab initio', 'a.k.a.'], 'legal theory': ['ab initio'], 'business sectors': ['ab initio', 'A/R'], 'courts and procedure': ['ab initio', 'a.k.a.', 'abandon'], 'criminal law and procedure': ['ab initio', 'a.k.a.', 'abet'], 'family law': ['ab initio', 'abandon'], 'legal education and practice': ['ab initio', 'a.k.a.', 'ABA', 'abandon'], 'wex definitions': ['ab initio', 'A/R', 'a.k.a.', 'ABA', 'aba

OPEN AI SEARCH ENGINE
- this is bypass the issue of too concise or too broad tags
- openai's vectorization system takes a look at all of the descriptions of each legal term & court cases & gets the most relevant to least relevant definitions & cases even if there isn't a specific tag for the particular search


i.e. 'robot' isn't a tag. Searching "robots" will bring any and every case/definition (depending on what part of the website you are on) related to this search - such as AI-related topics to least relevant of just normal tech topics

In [85]:
# embeddings = FakeEmbeddings(size=1536)
embeddings = OpenAIEmbeddings(openai_api_key = "INSERT OPENAI SECRET KEY HERE")
embedd_list = []

In [ ]:
for course in texts:
    embedd_list.append(embeddings.embed_query(course))

In [ ]:
embedd_list = np.array(embedd_list).astype("float32")
# embedd_list
index = faiss.IndexFlatL2(1536)
index.add(embedd_list)

In [67]:
def query_searches(query):
    x = 1
    query = np.array([embeddings.embed_query(query)]).astype("float32")
    k = 4 #get 4 most similar elements
    distances, indices = index.search(query, k)
    for inner_list in indices:
        for element in inner_list:
            print(str(x) + ". " + names[element])
            print("\t" + texts[element] + "\n\n")
            x+=1

EXAMPLE

In [65]:
query = "buisness related topics please"
query_searches(query)

1. ABA
	ABA is an abbreviation for American Bar Association (ABA).


2. A/R
	A/R is the abbreviation for accounts receivable.


3. award
	An award is defined as a final judgment or decision. Generally, an award will include a declaration that one party owes another a certain amount of money.  Although the term often refers to the decisions of arbitrators and juries, it can also refer to the decisions of judges.


4. abeyance
	An abeyance is a temporary suspension of activity while awaiting the resolution of some other proceeding without which the activity in abeyance cannot continue. In the case of a plea in abeyance, the defendant enters a conditional plea of guilty which is then held in abeyance subject to a set of conditions. If the defendant fulfills these conditions, the charges are dropped. If the defendant fails to complete the conditions, the guilty plea is entered. For example, a defendant may be required to undergo a drug addiction treatment program or have a guilty plea ente

In [ ]:
# Interactive Example

text = input("What would you like to search for?: ")
get_tags(text)